## Prepare Environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import torch

# Before running any code that may consume GPU memory
torch.cuda.empty_cache()

In [ ]:
#Download the required packages to prepare the environment
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install transformers[torch]
! pip install -U accelerate
! pip install -U transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ijexv4oi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ijexv4oi
  Resolved https://github.com/huggingface/transformers to commit a3cdff417bb3db94d5402dc009a07df97954a863
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9125860 sha256=281c719d3b2b9a2760df060598c589fcd5b4932407e7cc8a074c2ad9d09d33e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-e0fz37cl/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset, Audio, DatasetDict
import pandas as pd

# Define the paths
dataset_path = '/home/team1/dataset/hdrtna_tamazight'
tsv_file = f'{dataset_path}/train.tsv'
audio_folder = f'{dataset_path}/audio'

# Load the TSV file
data = pd.read_csv(tsv_file, sep='\t')

# Convert the DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(data)

# Define a function to load audio paths
def load_audio(example):
    example['audio'] = f"{audio_folder}/{example['audio']}"
    return example

# Apply the function to the dataset
dataset = dataset.map(load_audio)

# Load the audio data
dataset = dataset.cast_column("audio", Audio())

# Split dataset into train and test
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Create a DatasetDict
common_voice = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1
    })
})


In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Audio
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

# Define the new tokens for Tamazight (these are the ones used in the dataset, there are more)
new_tokens = ["Č", "Ḍ", "Ǧ", "Ḥ", "Ɣ", "Ṛ", "Ṣ", "Ṭ", "Ẓ", "Ɛ", "ɛ", "ẓ", "ṭ", "ṣ", "ṛ", "ɣ", "ḥ", "ǧ", "ḍ", "č"]

# Load the Whisper model and tokenizer
model_name = "openai/whisper-medium"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="fr", task="transcribe")


# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Load the processor
processor = WhisperProcessor.from_pretrained(model_name, language="fr", task="transcribe")

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch


common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names['train'], num_proc=2)



Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Data collator class
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
# Initialize data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Load evaluation metric
metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.generation_config.language = "fr"
model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

Embedding(51883, 1024)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./FineTune_Whispermodium_ar",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


KeyboardInterrupt: 